## Question2

### Smart Beta Strategy

Use machine learning models select factors and stocks to predict benchmark return

## 1. Load Data

Data is pulled out from JoinQuant. All the data is in Chinese stock market. 

In [5]:
auth('13637671410','zq13637671410')

auth success


In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.svm import SVR  
from sklearn.model_selection import GridSearchCV  
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from jqdatasdk import *

In [6]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

### 1) Creat functions extract financial ratio data from Joinquant

Selected factors: 

Valuation factors: total owner equities/market capitalization(B/M), basic earning per share(EPS), price earning ratio(PE Ratio); 

Growth factors: net profit/total owner equities(ROE), net profit/total assets(ROA), total profit/operating revenue(GP/R), net profit/operating revenue(P/R)

Capital structure factors: total liability/total assets(L/A), fixed assets/total assets(FAP), circulating market capitalization(CMV)

In [8]:
factors = ['m_cap','B/M','EPS','PEG','ROE','ROA','GP/R','P/R','L/A','FAP','CMV']

In [9]:
def get_factors(fdate,factors):
    ## pull out the factors in Shanghai Stock Exchange Market data
    stock_set = get_index_stocks('000001.XSHG',fdate)
    q = query(
        valuation.code,
        valuation.market_cap,
        balance.total_owner_equities/valuation.market_cap/100000000,
        income.basic_eps,
        valuation.pe_ratio,
        income.net_profit/balance.total_owner_equities,
        income.net_profit/balance.total_assets,
        income.total_profit/income.operating_revenue,
        income.net_profit/income.operating_revenue,
        balance.total_liability/balance.total_assets,
        balance.fixed_assets/balance.total_assets,
        valuation.circulating_market_cap
        ).filter(
        valuation.code.in_(stock_set),
        valuation.circulating_market_cap
    )
    fdf = get_fundamentals(q, date=fdate)
    fdf.index = fdf['code']
    fdf.columns = ['code'] + factors
    fdf = fdf.drop('code',axis=1)
    
    ## standardized factors except m_cap, m_cap is not in the model but just for ranking and extracting the top stocks. 
    fdf_standard = pd.DataFrame()
    fdf_standard['m_cap'] = fdf['m_cap']
    for i in range(1,fdf.shape[1]): 
        col = fdf.columns[i]
        fdf_standard[col] = (fdf[col] - fdf[col].mean())/fdf[col].std()
        
    return fdf_standard

In [10]:
## check a sample
fdf = get_factors('2018-12-01',factors)
fdf.head()

,m_cap,B/M,EPS,PEG,ROE,ROA,GP/R,P/R,L/A,FAP,CMV
code,,,,,,,,,,,
600000.XSHG,3143.6079,1.984293,1.376447,-0.084399,0.118240,-0.548678,0.128186,0.122015,2.016591,-1.156201,3.800616
600004.XSHG,209.6222,0.247869,-0.087386,-0.056103,-0.051952,-0.221850,0.042328,0.039278,-0.123894,3.341531,0.057836
600006.XSHG,76.6000,0.742517,-0.275057,-0.057018,-0.034849,-0.292729,-0.001901,0.011813,0.485515,-0.644283,-0.119957
600007.XSHG,142.8327,-0.439869,0.175353,-0.046390,0.077012,0.265706,0.107064,0.087774,-0.233304,-0.147087,-0.031433
600008.XSHG,196.1480,0.762670,-0.329482,-0.010706,-0.069130,-0.475047,0.030833,0.027988,1.046142,-0.579844,-0.000052


### 2) Get all dates from the selected time period

In [11]:
from dateutil.relativedelta import relativedelta
import datetime

In [12]:
## get monthly dates from 2013-01-01 to 2017-12-01
result = []
today = datetime.date(2017,12,1)
current = datetime.date(2013, 1, 1)    

while current <= today:
    result.append(current.strftime('%Y-%m-%d'))
    current += relativedelta(months=1)

### 3) Extract data and store them into a dictionary by dates

In [13]:
fdf_dict = {}
for x in result:
    fdf=get_factors(x,factors)
    fdf_dict[x] = fdf

In [14]:
## save data into csv files
for x in result:
    fdf_dict[x].to_csv('data/'+ x+'.csv')

In [15]:
## dates for calculating benchmark
result1 = []
today1 = datetime.date(2018,1,1)
current1 = datetime.date(2013, 1, 1)    

while current1 <= today1:
    result1.append(current1.strftime('%Y-%m-%d'))
    current1 += relativedelta(months=1)

### 4) Calculating Benchmark return for all dates

In [16]:
## use 000001.XSHG (Shanghai Exchange Stock Index) as the benchmark, and pull out the return
def caculate_benchmark_monthly_return(startdate,enddate,nextdate):

    close1 = get_price(['000001.XSHG'],startdate,enddate,'daily',['close'])['close']
    close2 = get_price(['000001.XSHG'],enddate, nextdate, 'daily',['close'])['close']
    benchmark_return = (close2.ix[0,:]/close1.ix[0,:]-1).sum()
    
    return benchmark_return

In [17]:
bench=pd.DataFrame()
for i in range(0,len(result1)-2):
    bench['return_'+result1[i+1]]=pd.Series(caculate_benchmark_monthly_return(result1[i],result1[i+1],result1[i+2]))

In [18]:
bench_r=bench.T
bench_r.columns = ['return']
print(bench_r.shape)
print(bench_r.head())

(59, 1)
                     return
return_2013-02-01  0.062375
return_2013-03-01 -0.024603
return_2013-04-01 -0.053024
return_2013-05-01 -0.026975
return_2013-06-01  0.057554


### 5) pull out Capitalization and extract top 300 stocks data according to the mean capitalization

In [19]:
## pull out capitalization
actual=pd.DataFrame()
mkt_cap=pd.DataFrame()
for x in result:
    df=fdf_dict[x]
    actual['CMV_'+x]=pd.Series(df['CMV'].fillna(0))
    mkt_cap['m_cap_'+x]=pd.Series(df['m_cap'].fillna(0))
mkt_cap.head()

,m_cap_2013-01-01,m_cap_2013-02-01,m_cap_2013-03-01,m_cap_2013-04-01,m_cap_2013-05-01,m_cap_2013-06-01,m_cap_2013-07-01,m_cap_2013-08-01,m_cap_2013-09-01,m_cap_2013-10-01,...,m_cap_2017-03-01,m_cap_2017-04-01,m_cap_2017-05-01,m_cap_2017-06-01,m_cap_2017-07-01,m_cap_2017-08-01,m_cap_2017-09-01,m_cap_2017-10-01,m_cap_2017-11-01,m_cap_2017-12-01
code,,,,,,,,,,,,,,,,,,,,,
600000.XSHG,1850.42,2229.0901,2083.5901,1874.67,1842.96,1953.02,1523.99,1490.41,1676.95,1882.14,...,3579.99,3461.0901,3288.1399,3631.0063,3555.1262,3774.3354,3588.8506,3777.6128,3680.7510,3789.3535
600004.XSHG,81.54,87.1700,85.5600,79.58,78.89,81.08,72.56,70.04,77.86,77.86,...,172.74,183.3200,184.4500,212.5466,263.4459,278.5305,276.8751,270.8741,295.2920,290.7395
600005.XSHG,279.60,299.7900,296.7600,276.57,260.42,267.49,225.09,230.14,237.20,242.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600006.XSHG,59.80,64.0000,62.8000,57.80,59.20,66.80,56.60,55.60,59.00,60.40,...,146.20,137.6000,126.2000,110.6000,117.0000,123.2000,118.8000,138.2000,127.0000,119.2000
600007.XSHG,115.74,114.2300,120.9700,114.23,104.86,112.92,92.67,92.37,110.70,113.52,...,200.15,201.5600,201.4600,206.9966,209.6155,198.6361,199.8449,189.8728,182.8218,175.3679


In [20]:
## calculate mean and rank
mkt_cap['mean'] = mkt_cap.mean(axis=1)
mkt_cap_s=mkt_cap.sort_index(by='mean',ascending=False)
slist = list(mkt_cap_s.index[:300])

In [21]:
## get top 300 data into a sub dictionary
sub_dict={}
for x in result:
    sub_dict[x]=fdf_dict[x].loc[slist]

### 6) calculate return for 300 stocks

In [22]:
def caculate_stock_monthly_return(stock,startdate,enddate,nextdate):

    close1 = get_price(stock,startdate,enddate,'daily',['close'])['close']
    close2 = get_price(stock,enddate, nextdate, 'daily',['close'])['close']
    stock_return = (close2.ix[0,:]/close1.ix[0,:]-1)
    return stock_return

In [23]:
r=caculate_stock_monthly_return(slist, '2013-01-01', '2013-02-01','2013-03-01')

In [24]:
stock_r=pd.DataFrame()
for i in range(0,len(result1)-2):
    stock_r['return_'+result1[i+1]]=pd.Series(caculate_stock_monthly_return(slist,result1[i],result1[i+1],result1[i+2]))

In [25]:
stock_r.head()

,return_2013-02-01,return_2013-03-01,return_2013-04-01,return_2013-05-01,return_2013-06-01,return_2013-07-01,return_2013-08-01,return_2013-09-01,return_2013-10-01,return_2013-11-01,...,return_2017-03-01,return_2017-04-01,return_2017-05-01,return_2017-06-01,return_2017-07-01,return_2017-08-01,return_2017-09-01,return_2017-10-01,return_2017-11-01,return_2017-12-01
601398.XSHG,0.055556,-0.061920,-0.026403,0.003390,0.033784,0.016340,-0.028939,-0.003311,-0.006645,-0.006689,...,0.026379,0.037383,-0.006757,0.102041,-0.012346,0.108333,0.043233,0.052252,-0.001712,-0.032590
601857.XSHG,0.026415,-0.030637,-0.035398,-0.030144,0.012162,-0.085447,0.051095,-0.031944,0.031564,-0.005563,...,-0.038508,-0.033792,-0.034974,0.020134,-0.009211,0.063745,-0.017478,-0.005083,0.052363,-0.027913
601939.XSHG,0.076923,-0.071429,-0.023669,0.027273,0.020649,-0.046243,0.003030,-0.018127,0.012308,0.006079,...,0.034549,0.020408,-0.016364,0.085028,0.000000,0.109029,0.006144,0.025954,0.004464,-0.010370
601288.XSHG,0.086957,-0.057778,-0.094340,0.010417,0.015464,-0.035533,-0.005263,-0.005291,0.021277,0.015625,...,0.024221,0.027027,-0.006579,0.076159,-0.015385,0.112500,-0.008427,0.022663,-0.013850,0.000000
601988.XSHG,0.070093,-0.056769,-0.027778,-0.019048,0.038835,-0.023364,-0.009569,-0.014493,0.049020,0.014019,...,0.034056,0.002994,-0.032836,0.049383,-0.011765,0.122024,0.061008,-0.020000,-0.051020,0.002688


## 2. Build Model

### 1) Variable selection - VIF

Randomly pick one date data, and check if there is any high correlated factors

In [26]:
df=get_factors('2015-01-01',factors).loc[slist]
#log_df=np.log(df)
#log_df=log_df.replace([np.inf, -np.inf], np.nan).fillna(0)
X = df[['B/M','EPS','PEG','ROE','ROA','P/R','L/A','FAP','CMV']].fillna(0)
Y = stock_r['return_2015-01-01'].fillna(0)

In [27]:
##testing multicolinearity of factors
vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns
vif.sort_values(by='VIF Factor', ascending=False)

,VIF Factor,features
4,1.763112,ROA
1,1.622888,EPS
5,1.482211,P/R
3,1.467764,ROE
6,1.266231,L/A
0,1.230189,B/M
8,1.227815,CMV
7,1.159436,FAP
2,1.016889,PEG


### 2) variable selection - multivariate linear regression

Randomly pick three dates data and run linear regression model, and remove insignificant factors

In [28]:
df=get_factors('2015-01-01',factors).loc[slist]
X = df[['B/M','EPS','PEG','ROE','ROA','P/R','L/A','FAP','CMV']].fillna(0)
mod=sm.OLS(Y,X).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return_2015-01-01   R-squared:                       0.554
Model:                            OLS   Adj. R-squared:                  0.541
Method:                 Least Squares   F-statistic:                     40.22
Date:                Fri, 14 Dec 2018   Prob (F-statistic):           3.70e-46
Time:                        13:16:20   Log-Likelihood:                 61.737
No. Observations:                 300   AIC:                            -105.5
Df Residuals:                     291   BIC:                            -72.14
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
B/M            0.0323      0.012      2.763      0.006       0.009       0.055
EPS            0.0110      0.010      1.061      0.290      -0.009       0.031
PEG            0.0195      0.016      1.206      0.229      -0.012       0.051
ROE            0.1754      0.030      5.909      0.000       0.117       0.234
ROA           -0.0359      0.012     -3.009      0.003      -0.059      -0.012
P/R            3.2173      0.254     12.661      0.000       2.717       3.717
L/A            0.0907      0.014      6.623      0.000       0.064       0.118
FAP            0.0007      0.012      0.061      0.951      -0.023       0.025
CMV            0.0157      0.007      2.177      0.030       0.002       0.030
==============================================================================
Omnibus:                       38.538   Durbin-Watson:                   1.667
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               62.885
Skew:                           0.763   Prob(JB):                     2.21e-14
Kurtosis:                       4.644   Cond. No.                         42.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [29]:
df=get_factors('2016-03-01',factors).loc[slist]
X = df[['B/M','EPS','PEG','ROE','ROA','P/R','L/A','FAP','CMV']].fillna(0)
mod=sm.OLS(Y,X).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return_2015-01-01   R-squared:                       0.535
Model:                            OLS   Adj. R-squared:                  0.521
Method:                 Least Squares   F-statistic:                     37.25
Date:                Fri, 14 Dec 2018   Prob (F-statistic):           1.42e-43
Time:                        13:16:25   Log-Likelihood:                 55.477
No. Observations:                 300   AIC:                            -92.95
Df Residuals:                     291   BIC:                            -59.62
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
B/M            0.0968      0.011      8.484      0.000       0.074       0.119
EPS            0.0231      0.013      1.777      0.077      -0.002       0.049
PEG            2.0339      0.303      6.718      0.000       1.438       2.630
ROE           -0.0248      0.241     -0.103      0.918      -0.500       0.450
ROA           -0.0197      0.060     -0.329      0.743      -0.138       0.098
P/R            0.1162      0.054      2.157      0.032       0.010       0.222
L/A            0.3472      0.095      3.657      0.000       0.160       0.534
FAP           -0.0226      0.012     -1.854      0.065      -0.047       0.001
CMV            0.0067      0.007      0.919      0.359      -0.008       0.021
==============================================================================
Omnibus:                       68.777   Durbin-Watson:                   1.762
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              156.271
Skew:                           1.118   Prob(JB):                     1.16e-34
Kurtosis:                       5.739   Cond. No.                         51.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
df=get_factors('2017-06-01',factors).loc[slist]
X = df[['B/M','EPS','PEG','ROE','ROA','P/R','L/A','FAP','CMV']].fillna(0)
mod=sm.OLS(Y,X).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      return_2015-01-01   R-squared:                       0.524
Model:                            OLS   Adj. R-squared:                  0.510
Method:                 Least Squares   F-statistic:                     35.63
Date:                Fri, 14 Dec 2018   Prob (F-statistic):           4.07e-42
Time:                        13:16:29   Log-Likelihood:                 51.944
No. Observations:                 300   AIC:                            -85.89
Df Residuals:                     291   BIC:                            -52.55
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
B/M            0.0777      0.012      6.739      0.000       0.055       0.100
EPS            0.0086      0.010      0.851      0.395      -0.011       0.028
PEG           -0.9279      0.229     -4.044      0.000      -1.380      -0.476
ROE           -0.4838      0.344     -1.406      0.161      -1.161       0.193
ROA            0.0703      0.032      2.177      0.030       0.007       0.134
P/R            0.0365      0.033      1.104      0.270      -0.029       0.101
L/A            0.1246      0.018      6.970      0.000       0.089       0.160
FAP            0.0103      0.012      0.878      0.381      -0.013       0.033
CMV            0.0138      0.007      1.989      0.048       0.000       0.027
==============================================================================
Omnibus:                       38.797   Durbin-Watson:                   1.663
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               72.101
Skew:                           0.715   Prob(JB):                     2.21e-16
Kurtosis:                       4.929   Cond. No.                         63.5
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
factor_list = ['B/M', 'PEG', 'P/R', 'L/A', 'CMV']

### 3) Machine Learning Models: SVR, Random Forest, Neural Network

In [32]:
reg=pd.DataFrame()
err_reg=pd.DataFrame()
mae_reg=pd.DataFrame()
rmse_reg=pd.DataFrame()

svrmod=pd.DataFrame()
err_svr=pd.DataFrame()
mae_svr=pd.DataFrame()
rmse_svr=pd.DataFrame()

rfmodel=pd.DataFrame()
err_rf=pd.DataFrame()
mae_rf=pd.DataFrame()
rmse_rf=pd.DataFrame()

NN=pd.DataFrame()
err_nn=pd.DataFrame()
mae_nn=pd.DataFrame()
rmse_nn=pd.DataFrame()

for i in range(1,60):
    x=result[i]
    df=sub_dict[x]
    #log_df=np.log(df)
    #log_df=log_df.replace([np.inf, -np.inf], np.nan).fillna(0)
    X = df[['B/M', 'PEG', 'P/R', 'L/A', 'CMV']].fillna(0)
    Y = stock_r['return_'+x].fillna(0)
    
    ## multivariate regression model
    mod=sm.OLS(Y,X)
    res=mod.fit()
    reg['return_'+x]=pd.Series(res.predict(X),index=Y.index)
    predictions0 = pd.DataFrame(res.predict(X))
    predictions0.index=Y.index
    predictions0.columns=['return']
    err_reg['err_'+x]=pd.Series(abs(predictions0['return'] - Y))# Print out the abs error
    mae_reg['mae_'+x]=pd.Series((abs(predictions0['return'] - Y)).mean())# Print out the mean absolute error (mae)
    rmse_reg['rmse_'+x]=pd.Series( math.sqrt(((predictions0['return'] - Y)**2).mean()) )# Calculate rmse
    
    ## SVR model
    svr = SVR(kernel='rbf', gamma=0.1) 
    model = svr.fit(X, Y)
    svrmod['return_'+x]=pd.Series(model.predict(X),index=Y.index)
    predictions1 = pd.DataFrame(model.predict(X))
    predictions1.index=Y.index
    predictions1.columns=['return']
    err_svr['mae_'+x]=pd.Series(abs(predictions1['return'] - Y))
    mae_svr['mae_'+x]=pd.Series(abs(predictions1['return'] - Y).mean())# Print out the mean absolute error (mae)
    rmse_svr['rmse_'+x]=pd.Series( math.sqrt(((predictions1['return'] - Y)**2).mean()) )# Calculate rmse
    
    ## random forest model
    rf = RandomForestRegressor(n_estimators = 100, random_state = 42)
    rfmod=rf.fit(X, Y)
    rfmodel['return_'+x]=pd.Series(rfmod.predict(X),index=Y.index)
    predictions2 = pd.DataFrame(rfmod.predict(X))
    predictions2.index=Y.index
    predictions2.columns=['return']
    err_rf['err_'+x]=pd.Series(abs(predictions2['return'] - Y))# Print out the abs error
    mae_rf['mae_'+x]=pd.Series(abs(predictions2['return'] - Y).mean())# Print out the mean absolute error (mae)
    rmse_rf['rmse_'+x]=pd.Series( math.sqrt(((predictions2['return'] - Y)**2).mean()) )# Calculate rmse
  
    ## neural network model
    clf = MLPRegressor(hidden_layer_sizes=(3), activation='tanh', solver='lbfgs')
    nnmod=clf.fit(X, Y) 
    NN['return_'+x]=pd.Series(nnmod.predict(X),index=Y.index)
    predictions3 = pd.DataFrame(nnmod.predict(X))
    predictions3.index=Y.index
    predictions3.columns=['return']
    err_nn['err_'+x]=pd.Series(abs(predictions3['return'] - Y))# Print out the abs error
    mae_nn['mae_'+x]=pd.Series(abs(predictions3['return'] - Y).mean())# Print out the mean absolute error (mae)
    rmse_nn['rmse_'+x]=pd.Series( math.sqrt(((predictions3['return'] - Y)**2).mean()) )# Calculate rmse
  

### 4) Select Model based on MAE/RMSE

In [34]:
mae_reg_m=mae_reg.mean(axis=1)
rmse_reg_m=rmse_reg.mean(axis=1)
print('MAE:' ,mae_reg_m[0],'RMSE:',rmse_reg_m[0])

MAE: 0.07051887786571352 RMSE: 0.10378027454721404


In [35]:
mae_svr_m=mae_svr.mean(axis=1)
rmse_svr_m=rmse_svr.mean(axis=1)
print('MAE:' ,mae_svr_m[0],'RMSE:',rmse_svr_m[0])

MAE: 0.06886984555245142 RMSE: 0.09800363791055651


In [36]:
mae_rf_m=mae_rf.mean(axis=1)
rmse_rf_m=rmse_rf.mean(axis=1)
print('MAE:' ,mae_rf_m[0],'RMSE:',rmse_rf_m[0])

MAE: 0.025735978884134352 RMSE: 0.03834188024272441


In [37]:
mae_nn_m=mae_nn.mean(axis=1)
rmse_nn_m=rmse_nn.mean(axis=1)
print('MAE:' ,mae_nn_m[0],'RMSE:',rmse_nn_m[0])

MAE: 0.06461351468311773 RMSE: 0.09320297229151003


According to MAE and RMSE, SVR, RF, and NN are all better than linear regression model, and random forest is the best one.

In [38]:
feature_importances = pd.DataFrame(rf.feature_importances_,
                                   index = X.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)
print(feature_importances)

     importance
CMV    0.271761
P/R    0.212537
PEG    0.186464
L/A    0.169475
B/M    0.159763


## 3. Construct Portfolio

### 1) equally weighted

select stocks according to the mean of the error, and select top 10 stocks

In [39]:
## tracking error -- regression
err_reg['mean']=err_reg.mean(axis=1)
err_reg_s=err_reg.sort_index(by='mean')
stockset_reg = list(err_reg_s.index[:10])
port_reg=reg.loc[stockset_reg]
reg_r=port_reg.mean() ##assuming equally weighted
tracking_err_reg = np.std(reg_r-bench_r['return'])
print(tracking_err_reg)

0.024911550234833282


In [40]:
## tracking error -- SVR
err_svr['mean']=err_svr.mean(axis=1)
err_svr_s=err_svr.sort_index(by='mean')
stockset_svr = list(err_svr_s.index[:10])
port_svr=svrmod.loc[stockset_svr]
svr_r=port_svr.mean()
tracking_err_svr = np.std(svr_r-bench_r['return'])
print(tracking_err_svr)

0.027717865828212847


In [41]:
## tracking error -- rf
err_rf['mean']=err_rf.mean(axis=1)
err_rf_s=err_rf.sort_index(by='mean')
stockset_rf = list(err_rf_s.index[:10])
port_rf=rfmodel.loc[stockset_rf]
rf_r=port_rf.mean()
tracking_err_rf = np.std(rf_r-bench_r['return'])
print(tracking_err_rf)

0.03457203611478055


In [42]:
## tracking error -- nn
err_nn['mean']=err_nn.mean(axis=1)
err_nn_s=err_nn.sort_index(by='mean')
stockset_nn = list(err_nn_s.index[:10])
port_nn=NN.loc[stockset_nn]
nn_r=port_nn.mean()
tracking_err_nn = np.std(nn_r-bench_r['return'])
print(tracking_err_nn)

0.029250007263918215


### 2) Portfolio Optimization (using random forest based on the model selection in step 2)

In [43]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

In [44]:
stockset_rf

['601988.XSHG',
 '600015.XSHG',
 '601398.XSHG',
 '601818.XSHG',
 '601857.XSHG',
 '601288.XSHG',
 '600000.XSHG',
 '600377.XSHG',
 '600660.XSHG',
 '601328.XSHG']

In [46]:
# Read in price data
close = get_price(stockset_rf,'2013-01-01','2017-12-01','daily',['close'])['close']

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(close)
S = risk_models.sample_cov(close)

# Optimise for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
print(raw_weights)


{'601988.XSHG': 0.0, '600015.XSHG': 1.9619569430550897e-17, '601398.XSHG': 0.148821974294187, '601818.XSHG': 1.7150700570641979e-16, '601857.XSHG': 0.0, '601288.XSHG': 0.0, '600000.XSHG': 0.13171356606143864, '600377.XSHG': 0.16617442132400467, '600660.XSHG': 0.5532900383203697, '601328.XSHG': 7.09015946811154e-18}


In [47]:
ef.portfolio_performance(verbose=True)

Expected annual return: 27.8%
Annual volatility: 24.9%
Sharpe Ratio: 1.03


(0.2776214538745265, 0.2491072301814984, 1.0267141455474125)

In [48]:
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
ef.efficient_return(target_return=0.1, market_neutral=True)

{'600000.XSHG': 0.13846958037790516,
 '600015.XSHG': 0.08443204675672403,
 '600377.XSHG': 0.012096345774621303,
 '600660.XSHG': 0.2316302243960177,
 '601288.XSHG': -0.13720742979738948,
 '601328.XSHG': -0.1284996066992483,
 '601398.XSHG': 0.089980900221894,
 '601818.XSHG': -0.10262605977218188,
 '601857.XSHG': -0.3564112400023143,
 '601988.XSHG': 0.16813523874397174}

In [51]:
w=pd.DataFrame(raw_weights ,index=[0])

In [52]:
w.T[0]

601988.XSHG    0.000000e+00
600015.XSHG    1.961957e-17
601398.XSHG    1.488220e-01
601818.XSHG    1.715070e-16
601857.XSHG    0.000000e+00
601288.XSHG    0.000000e+00
600000.XSHG    1.317136e-01
600377.XSHG    1.661744e-01
600660.XSHG    5.532900e-01
601328.XSHG    7.090159e-18
Name: 0, dtype: float64

In [53]:
p_r=pd.DataFrame()
for i in range(1,60):
    x=result[i]
    p_r['return_'+x]=port_rf['return_'+x].dot(w.T)
pr=p_r.T
pr.columns = ['return']

In [54]:
te=np.std(pr['return']-bench_r['return'])
te

0.036576165848521545

### 3) Portfolio Optimization (using SVR)

In [55]:
stockset_svr

['600004.XSHG',
 '600015.XSHG',
 '600811.XSHG',
 '601818.XSHG',
 '600219.XSHG',
 '600350.XSHG',
 '600875.XSHG',
 '600377.XSHG',
 '600320.XSHG',
 '600548.XSHG']

In [56]:
# Read in price data
close = get_price(stockset_svr,'2013-01-01','2017-12-01','daily',['close'])['close']

# Calculate expected returns and sample covariance
mu = expected_returns.mean_historical_return(close)
S = risk_models.sample_cov(close)

# Optimise for maximal Sharpe ratio
ef = EfficientFrontier(mu, S)
raw_weights = ef.max_sharpe()
print(raw_weights)


{'600004.XSHG': 0.43535688564846436, '600015.XSHG': 0.17964098611968793, '600811.XSHG': 1.9656585387162195e-16, '601818.XSHG': 2.387413183813081e-16, '600219.XSHG': 1.5563722186029416e-16, '600350.XSHG': 0.0, '600875.XSHG': 0.0, '600377.XSHG': 0.06216637294036528, '600320.XSHG': 0.0, '600548.XSHG': 0.32283575529148256}


In [58]:
ef.portfolio_performance(verbose=True)

Expected annual return: 29.4%
Annual volatility: 28.6%
Sharpe Ratio: 0.95


(0.2943478447698406, 0.2858183733181779, 0.953269888825544)

In [59]:
ef = EfficientFrontier(mu, S, weight_bounds=(0, 1))
ef.efficient_return(target_return=0.1, market_neutral=True)

{'600004.XSHG': 0.2571037311521381,
 '600015.XSHG': 0.11769102660942511,
 '600219.XSHG': -0.04069176213102373,
 '600320.XSHG': 0.023240705928103494,
 '600350.XSHG': 0.05734906844965445,
 '600377.XSHG': -0.07061354872147159,
 '600548.XSHG': 0.1332736079214702,
 '600811.XSHG': -0.13071419656785926,
 '600875.XSHG': -0.21084992976307507,
 '601818.XSHG': -0.13578870287736167}

In [60]:
w=pd.DataFrame(raw_weights ,index=[0])

In [61]:
w.T

,0
600004.XSHG,4.353569e-01
600015.XSHG,1.796410e-01
600811.XSHG,1.965659e-16
601818.XSHG,2.387413e-16
600219.XSHG,1.556372e-16
600350.XSHG,0.000000e+00
600875.XSHG,0.000000e+00
600377.XSHG,6.216637e-02
600320.XSHG,0.000000e+00
600548.XSHG,3.228358e-01


In [62]:
p_r=pd.DataFrame()
for i in range(1,60):
    x=result[i]
    p_r['return_'+x]=port_svr['return_'+x].dot(w.T)
pr=p_r.T
pr.columns = ['return']

In [63]:
te=np.std(pr['return']-bench_r['return'])
te

0.032524841223821724

## 4. Conclusion

Based on the model's performance, Random Forest model is the best model on predict the stock's returns according to the MAE and RMSE, but based on the tracking error, which is our final goal, using SVR to select top 10 stocks and construct portfolio both equally weight or using optimized weight has a smaller tracking error than random forest. And the MAE and RMSE are not as big as linear regression, so I recommend to use SVR and equally weight stocks as my strategy for portfolio construction. 